In [1]:
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import os

Se cargan todos los archivos .json del directorio ../data y todos sus subdirectorios respentando orden original.

In [3]:
# Directorio raíz donde buscar archivos JSON
root_folder = "../data/"  # Ajusta según tu estructura

# Lista para almacenar rutas de archivos JSON
all_files = []

# Recorrer todos los directorios y subdirectorios
for dirpath, _, filenames in os.walk(root_folder):
    # Buscar archivos JSON en cada directorio
    json_files = glob.glob(os.path.join(dirpath, "*.json"))
    all_files.extend(json_files)

# Mostrar cantidad de archivos encontrados
print(f"Se encontraron {len(all_files)} archivos JSON.")
print(all_files[:5])  # Muestra los primeros archivos para verificar


Se encontraron 625 archivos JSON.
['../data/google_maps\\metadata-sitios\\1.json', '../data/google_maps\\metadata-sitios\\10.json', '../data/google_maps\\metadata-sitios\\11.json', '../data/google_maps\\metadata-sitios\\2.json', '../data/google_maps\\metadata-sitios\\3.json']


In [7]:
import json
import pandas as pd
import os
import glob

# Directorios raíz donde buscar archivos JSON
metadata_dir = "../data/google_maps/metadata-sitios"
reviews_root_dir = "../data/google_maps/reviews-estados"

metadata_files = glob.glob(os.path.join(metadata_dir, "*.json"))
reviews_files = []
for dirpath, _, filenames in os.walk(reviews_root_dir):
    for filename in filenames:
        if filename.endswith(".json"):
            reviews_files.append(os.path.join(dirpath, filename))

print(f"Se encontraron {len(metadata_files)} archivos de metadatos de sitios.")
print(f"Se encontraron {len(reviews_files)} archivos de reseñas.")

lista_metadata = []
metadata_errores = []
lista_reviews = []
reviews_errores = []

def cargar_json_robustamente(archivo, lista_destino, lista_errores, estado=None):
    """
    Intenta cargar uno o más objetos JSON desde un archivo, manejando errores comunes.
    """
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            # Reemplazar saltos de línea de Windows (CR LF) con saltos de línea Unix (\n)
            contenido = contenido.replace('\r\n', '\n')
            try:
                # Intenta cargar como un solo objeto JSON
                data = json.loads(contenido)
                if estado:
                    data['state'] = estado
                lista_destino.append(data)
            except json.JSONDecodeError as e_single:
                print(f"Error al decodificar JSON único (UTF-8) en {archivo}: {e_single}")
                try:
                    # Intenta cargar como múltiples objetos JSON concatenados
                    objetos = contenido.strip().split('\n')
                    for i, obj_str in enumerate(objetos):
                        if obj_str.strip():
                            try:
                                data = json.loads(obj_str)
                                if estado:
                                    data['state'] = estado
                                lista_destino.append(data)
                            except json.JSONDecodeError as e_multi:
                                print(
                                    f"Error al decodificar objeto JSON individual (UTF-8) en {archivo}, línea {i + 1}: {e_multi} - Se ignorará la línea.")
                                lista_errores.append(archivo)
                except Exception as e_multi_split:
                    print(
                        f"Error al intentar dividir el archivo en múltiples objetos JSON: {e_multi_split}")
                    lista_errores.append(archivo)
    except FileNotFoundError:
        print(f"Error: El archivo {archivo} no fue encontrado.")
        lista_errores.append(archivo)
    except Exception as e_general:
        print(f"Error general al leer el archivo {archivo}: {e_general}")
        lista_errores.append(archivo)

# Pedir al usuario el número de archivo a procesar
while True:
    try:
        metadata_index = int(input(
            f"Ingrese el número de archivo a procesar de metadatos (1-{len(metadata_files)}, 0 para todos): "))
        if 0 <= metadata_index <= len(metadata_files):
            break
        else:
            print("Número de archivo inválido. Por favor, ingrese un número dentro del rango válido.")
    except ValueError:
        print("Entrada inválida. Por favor, ingrese un número entero.")

while True:
    try:
        reviews_index = int(input(
            f"Ingrese el número de archivo a procesar de reseñas (1-{len(reviews_files)}, 0 para todos): "))
        if 0 <= reviews_index <= len(reviews_files):
            break
        else:
            print("Número de archivo inválido. Por favor, ingrese un número dentro del rango válido.")
    except ValueError:
        print("Entrada inválida. Por favor, ingrese un número entero.")

# Cargar metadatos de sitios
if metadata_index == 0:
    for archivo in metadata_files:
        cargar_json_robustamente(archivo, lista_metadata, metadata_errores)
else:
    archivo = metadata_files[metadata_index - 1]
    cargar_json_robustamente(archivo, lista_metadata, metadata_errores)

# Cargar reseñas
if reviews_index == 0:
    for archivo in reviews_files:
        cargar_json_robustamente(archivo, lista_reviews, reviews_errores)
else:
    archivo = reviews_files[reviews_index - 1]
    cargar_json_robustamente(archivo, lista_reviews, reviews_errores)

print("\n--- Metadatos de Sitios ---")
if lista_metadata:
    df_metadata = pd.DataFrame(lista_metadata)
    print(df_metadata.head())
    print(df_metadata.describe())
    # ... (Aquí puedes agregar EDA específico para metadatos)
else:
    print("No se cargaron datos de metadatos de sitios.")
if metadata_errores:
    print("\nArchivos de metadatos con errores:")
    for archivo in metadata_errores:
        print(archivo)

print("\n--- Reseñas ---")
if lista_reviews:
    df_reviews = pd.DataFrame(lista_reviews)
    print(df_reviews.head())
    print(df_reviews.describe())
    # ... (Aquí puedes agregar EDA específico para reseñas)
else:
    print("No se cargaron datos de reseñas.")
if reviews_errores:
    print("\nArchivos de reseñas con errores:")
    for archivo in reviews_errores:
        print(archivo)



Se encontraron 11 archivos de metadatos de sitios.
Se encontraron 611 archivos de reseñas.
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\1.json: Extra data: line 2 column 1 (char 1055)
Error al decodificar JSON único (UTF-8) en ../data/google_maps/reviews-estados\review-Alabama\1.json: Extra data: line 2 column 1 (char 291)

--- Metadatos de Sitios ---
               name                                            address  \
0   Porter Pharmacy  Porter Pharmacy, 129 N Second St, Cochran, GA ...   
1      City Textile  City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2      San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3      Nova Fabrics  Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4  Nobel Textile Co  Nobel Textile Co, 719 E 9th St, Los Angeles, C...   

                                 gmap_id description   latitude   longitude  \
0  0x88f16e41928ff687:0x883dad4fd048e8f8        None  32.388300  -83.357100   
1  0x80c2

In [1]:
import json
import pandas as pd
import os
import glob

# Directorios raíz donde buscar archivos JSON
metadata_dir = "../data/google_maps/metadata-sitios"
reviews_root_dir = "../data/google_maps/reviews-estados"
parquet_dir = "../data/parquet_files"
os.makedirs(parquet_dir, exist_ok=True)  # Crea el directorio si no existe

metadata_files = glob.glob(os.path.join(metadata_dir, "*.json"))
reviews_files = []
for dirpath, _, filenames in os.walk(reviews_root_dir):
    for filename in filenames:
        if filename.endswith(".json"):
            reviews_files.append(os.path.join(dirpath, filename))

print(f"Se encontraron {len(metadata_files)} archivos de metadatos de sitios.")
print(f"Se encontraron {len(reviews_files)} archivos de reseñas.")

metadata_errores = []
reviews_errores = []

def cargar_json_robustamente(archivo, estado=None):
    """
    Intenta cargar uno o más objetos JSON desde un archivo, manejando errores comunes.
    Args:
        archivo (str): La ruta al archivo JSON.
        estado (str, opcional): El estado asociado con los datos (para archivos de reseñas).
    Returns:
        list: Una lista de diccionarios con los datos cargados, o None si hay un error.
    """
    lista_datos = []
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            # Reemplazar saltos de línea de Windows (CR LF) con saltos de línea Unix (\n)
            contenido = contenido.replace('\r\n', '\n')
            try:
                # Intenta cargar como un solo objeto JSON
                data = json.loads(contenido)
                if estado:
                    data['state'] = estado  # Agrega el estado si está disponible
                lista_datos.append(data)
            except json.JSONDecodeError as e_single:
                print(f"Error al decodificar JSON único (UTF-8) en {archivo}: {e_single}")
                try:
                    # Intenta cargar como múltiples objetos JSON concatenados
                    objetos = contenido.strip().split('\n')
                    for i, obj_str in enumerate(objetos):
                        if obj_str.strip():
                            try:
                                data = json.loads(obj_str)
                                if estado:
                                    data['state'] = estado  # Agrega el estado a cada objeto
                                lista_datos.append(data)
                            except json.JSONDecodeError as e_multi:
                                print(
                                    f"Error al decodificar objeto JSON individual (UTF-8) en {archivo}, línea {i + 1}: {e_multi} - Se ignorará la línea.")
                                lista_errores.append(archivo)
                except Exception as e_multi_split:
                    print(
                        f"Error al intentar dividir el archivo en múltiples objetos JSON: {e_multi_split}")
                    lista_errores.append(archivo)
    except FileNotFoundError:
        print(f"Error: El archivo {archivo} no fue encontrado.")
        lista_errores.append(archivo)
    except Exception as e_general:
        print(f"Error general al leer el archivo {archivo}: {e_general}")
        lista_errores.append(archivo)
    return lista_datos

def crear_parquet_con_manejo_de_errores(lista_datos, nombre_archivo, parquet_dir):
    """
    Crea un archivo Parquet a partir de una lista de datos, manejando errores.
    Args:
        lista_datos (list): Lista de diccionarios con los datos.
        nombre_archivo (str): Nombre del archivo Parquet a crear.
        parquet_dir (str): Directorio donde se guardará el archivo Parquet.
    """
    if lista_datos:
        try:
            df = pd.DataFrame(lista_datos)
            parquet_file = os.path.join(parquet_dir, nombre_archivo)
            df.to_parquet(parquet_file)
            print(f"Guardado en: {parquet_file}")
        except Exception as e:
            print(f"Error al crear el archivo Parquet {nombre_archivo}: {e}")
    else:
        print(f"No hay datos para guardar en {nombre_archivo}")

# Procesar metadatos secuencialmente
for archivo in metadata_files:
    lista_metadata = cargar_json_robustamente(archivo)
    if lista_metadata:
        crear_parquet_con_manejo_de_errores(lista_metadata, f"metadata_sitios_{os.path.basename(archivo).replace('.json', '.parquet')}", parquet_dir)
    else:
        print(f"No se pudieron cargar datos de {archivo}")

# Procesar reseñas secuencialmente
for archivo in reviews_files:
    estado = os.path.basename(os.path.dirname(archivo))
    lista_reviews = cargar_json_robustamente(archivo, estado=estado)
    if lista_reviews:
        crear_parquet_con_manejo_de_errores(lista_reviews, f"reviews_{os.path.basename(archivo).replace('.json', '.parquet')}", parquet_dir)
    else:
        print(f"No se pudieron cargar datos de {archivo}")

if metadata_errores:
    print("\nArchivos de metadatos con errores:")
    for archivo in metadata_errores:
        print(archivo)

if reviews_errores:
    print("\nArchivos de reseñas con errores:")
    for archivo in reviews_errores:
        print(archivo)


Se encontraron 11 archivos de metadatos de sitios.
Se encontraron 611 archivos de reseñas.
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\1.json: Extra data: line 2 column 1 (char 1055)
Guardado en: ../data/parquet_files\metadata_sitios_1.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\10.json: Extra data: line 2 column 1 (char 1055)
Guardado en: ../data/parquet_files\metadata_sitios_10.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\11.json: Extra data: line 2 column 1 (char 1055)
Guardado en: ../data/parquet_files\metadata_sitios_11.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\2.json: Extra data: line 2 column 1 (char 1055)
Guardado en: ../data/parquet_files\metadata_sitios_2.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\3.json: Extra data: line 2 column 1 (char 1055)
Guardado en: ../data/parqu

In [8]:
import json
import pandas as pd
import os
import glob
import pyarrow.parquet as pq  # Importa pyarrow.parquet
import pyarrow as pa

# Directorios raíz donde buscar archivos JSON
metadata_dir = "../data/google_maps/metadata-sitios"
reviews_root_dir = "../data/google_maps/reviews-estados"
parquet_dir = "../data/parquet_files"
os.makedirs(parquet_dir, exist_ok=True)  # Crea el directorio si no existe

metadata_files = glob.glob(os.path.join(metadata_dir, "*.json"))
reviews_files = []
for dirpath, _, filenames in os.walk(reviews_root_dir):
    for filename in filenames:
        if filename.endswith(".json"):
            reviews_files.append(os.path.join(dirpath, filename))

print(f"Se encontraron {len(metadata_files)} archivos de metadatos de sitios.")
print(f"Se encontraron {len(reviews_files)} archivos de reseñas.")

metadata_errores = []
reviews_errores = []

def cargar_json_robustamente(archivo, estado=None):
    """
    Intenta cargar uno o más objetos JSON desde un archivo, manejando errores comunes.
    Args:
        archivo (str): La ruta al archivo JSON.
        estado (str, opcional): El estado asociado con los datos (para archivos de reseñas).
    Returns:
        list: Una lista de diccionarios con los datos cargados, o None si hay un error.
    """
    lista_datos = []
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            # Reemplazar saltos de línea de Windows (CR LF) con saltos de línea Unix (\n)
            contenido = contenido.replace('\r\n', '\n')
            try:
                # Intenta cargar como un solo objeto JSON
                data = json.loads(contenido)
                if estado:
                    data['state'] = estado  # Agrega el estado si está disponible
                lista_datos.append(data)
            except json.JSONDecodeError as e_single:
                print(f"Error al decodificar JSON único (UTF-8) en {archivo}: {e_single}")
                try:
                    # Intenta cargar como múltiples objetos JSON concatenados
                    objetos = contenido.strip().split('\n')
                    for i, obj_str in enumerate(objetos):
                        if obj_str.strip():
                            try:
                                data = json.loads(obj_str)
                                if estado:
                                    data['state'] = estado  # Agrega el estado a cada objeto
                                lista_datos.append(data)
                            except json.JSONDecodeError as e_multi:
                                print(
                                    f"Error al decodificar objeto JSON individual (UTF-8) en {archivo}, línea {i + 1}: {e_multi} - Se ignorará la línea.")
                                lista_errores.append(archivo)
                except Exception as e_multi_split:
                    print(
                        f"Error al intentar dividir el archivo en múltiples objetos JSON: {e_multi_split}")
                    lista_errores.append(archivo)
    except FileNotFoundError:
        print(f"Error: El archivo {archivo} no fue encontrado.")
        lista_errores.append(archivo)
    except Exception as e_general:
        print(f"Error general al leer el archivo {archivo}: {e_general}")
        lista_errores.append(archivo)
    return lista_datos

def json_a_parquet(archivo_json, archivo_parquet, estado=None):
    """
    Convierte un archivo JSON a un archivo Parquet.
    Args:
        archivo_json (str): La ruta al archivo JSON de entrada.
        archivo_parquet (str): La ruta al archivo Parquet de salida.
        estado (str, opcional): El estado asociado con los datos (para archivos de reseñas).
    """
    datos = cargar_json_robustamente(archivo_json, estado=estado)
    if datos:
        try:
            df = pd.DataFrame(datos)
            df.to_parquet(archivo_parquet)
            print(f"Convertido: {archivo_json} a {archivo_parquet}")
        except Exception as e:
            print(f"Error al convertir {archivo_json} a Parquet: {e}")
            return False  # Indica que hubo un error
        return True  # Indica que la conversión fue exitosa
    else:
        print(f"No se pudieron cargar datos de {archivo_json}")
        return False  # Indica que hubo un error


def combinar_parquet(archivos_parquet, archivo_salida):
    """
    Combina múltiples archivos Parquet en uno solo, compatible con versiones anteriores de pyarrow.
    Args:
        archivos_parquet (list): Lista de rutas a los archivos Parquet de entrada.
        archivo_salida (str): La ruta al archivo Parquet de salida combinado.
    """
    if archivos_parquet:
        try:
            # Leer el primer archivo Parquet para inicializar la tabla combinada
            combined_table = pq.read_table(archivos_parquet[0])
            # Iterar sobre los archivos restantes y concatenarlos
            for archivo in archivos_parquet[1:]:
                table = pq.read_table(archivo)
                # Unificar esquemas antes de concatenar
                combined_table = combined_table.cast(table.schema)
                combined_table = pa.concat_tables([combined_table, table])
            # Escribir la tabla combinada al archivo de salida
            pq.write_table(combined_table, archivo_salida)
            print(f"Combinados: {len(archivos_parquet)} archivos en {archivo_salida}")
        except Exception as e:
            print(f"Error al combinar archivos Parquet: {e}")
    else:
        print("No hay archivos Parquet para combinar.")

# Convertir metadatos a Parquet individualmente
archivos_parquet_metadata = []
for archivo in metadata_files:
    archivo_parquet = os.path.join(parquet_dir,
                                   f"metadata_sitios_{os.path.basename(archivo).replace('.json', '.parquet')}")
    if json_a_parquet(archivo, archivo_parquet):
        archivos_parquet_metadata.append(archivo_parquet)

# Combinar archivos Parquet de metadatos
archivo_salida_metadata = os.path.join(parquet_dir, "metadata_sitios_All.parquet")
if archivos_parquet_metadata:
    combinar_parquet(archivos_parquet_metadata, archivo_salida_metadata)
    # Eliminar archivos Parquet de metadatos individuales después de combinarlos
    for archivo_para_eliminar in archivos_parquet_metadata:
        try:
            os.remove(archivo_para_eliminar)
            print(f"Eliminado: {archivo_para_eliminar}")
        except Exception as e:
            print(f"Error al eliminar {archivo_para_eliminar}: {e}")
else:
    print("No hay archivos de metadatos para combinar.")

# Convertir reseñas a Parquet individualmente
archivos_parquet_reviews = []
estados_procesados = set()  # Para evitar procesar el mismo estado múltiples veces
for archivo in reviews_files:
    estado = os.path.basename(os.path.dirname(archivo)).replace("review-", "")  # Obtiene el estado del directorio
    archivo_parquet = os.path.join(parquet_dir,
                                   f"reviews_{estado}_{os.path.basename(archivo).replace('.json', '.parquet')}")  # Incluye el estado en el nombre del archivo
    if json_a_parquet(archivo, archivo_parquet, estado=estado):
        archivos_parquet_reviews.append(archivo_parquet)
    estados_procesados.add(estado)

# Combinar archivos Parquet por estado
for estado in estados_procesados:
    archivos_para_combinar = [archivo for archivo in archivos_parquet_reviews if f"reviews_{estado}_" in archivo]
    if archivos_para_combinar:
        archivo_salida = os.path.join(parquet_dir, f"reviews_{estado}_All.parquet")
        combinar_parquet(archivos_para_combinar, archivo_salida)
        # Eliminar archivos Parquet individuales después de combinarlos
        for archivo_para_eliminar in archivos_para_combinar:
            try:
                os.remove(archivo_para_eliminar)
                print(f"Eliminado: {archivo_para_eliminar}")
            except Exception as e:
                print(f"Error al eliminar {archivo_para_eliminar}: {e}")

if metadata_errores:
    print("\nArchivos de metadatos con errores:")
    for archivo in metadata_errores:
        print(archivo)

if reviews_errores:
    print("\nArchivos de reseñas con errores:")
    for archivo in reviews_errores:
        print(archivo)


Se encontraron 11 archivos de metadatos de sitios.
Se encontraron 611 archivos de reseñas.
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\1.json: Extra data: line 2 column 1 (char 1055)
Convertido: ../data/google_maps/metadata-sitios\1.json a ../data/parquet_files\metadata_sitios_1.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\10.json: Extra data: line 2 column 1 (char 1055)
Convertido: ../data/google_maps/metadata-sitios\10.json a ../data/parquet_files\metadata_sitios_10.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\11.json: Extra data: line 2 column 1 (char 1055)
Convertido: ../data/google_maps/metadata-sitios\11.json a ../data/parquet_files\metadata_sitios_11.parquet
Error al decodificar JSON único (UTF-8) en ../data/google_maps/metadata-sitios\2.json: Extra data: line 2 column 1 (char 1055)
Convertido: ../data/google_maps/metadata-sitios\2.json a ../data/parquet_files\m

#Analizando archivos .parquet generados

Tener instalado tabulate:

```
pip install tabulate
```

In [ ]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# Agrega esta línea para que los gráficos se muestren en Jupyter Notebook
# Si estás ejecutando el script en VS Code, esta línea no afectará el comportamiento.
%matplotlib inline

parquet_dir = "../data/parquet_files"

def realizar_eda_parquet(directorio):
    """
    Realiza un Análisis Exploratorio de Datos (EDA) básico en archivos Parquet.

    Args:
        directorio (str): Directorio que contiene los archivos Parquet.
    """
    # Obtener todos los archivos Parquet en el directorio
    archivos_parquet = glob.glob(os.path.join(directorio, "*.parquet"))

    if not archivos_parquet:
        print("No se encontraron archivos Parquet en el directorio especificado.")
        return

    for archivo in archivos_parquet:
        print(f"\nAnálisis del archivo: {archivo}")
        try:
            # Leer el archivo Parquet en un DataFrame de pandas
            df = pd.read_parquet(archivo)

            # Mostrar información general del DataFrame
            print("\nInformación general del DataFrame:")
            df.info()

            # Mostrar las primeras filas del DataFrame
            print("\nPrimeras 5 filas del DataFrame:")
            # Convertir cualquier numpy.ndarray a lista antes de imprimir
            head_df = df.head().copy()  # Crear una copia para no modificar el original
            for col in head_df.columns:
                if isinstance(head_df[col].iloc[0], pd.Series):
                    head_df[col] = head_df[col].apply(lambda x: x.tolist() if isinstance(x, pd.Series) else x)
                elif isinstance(head_df[col].iloc[0], dict):
                    head_df[col] = head_df[col].apply(lambda x: {k: (v.tolist() if isinstance(v, pd.Series) else v) for k, v in x.items()} if isinstance(x, dict) else x)
            print(head_df.to_markdown(index=False, numalign="left", stralign="left"))

            # Mostrar estadísticas descriptivas de las columnas numéricas
            print("\nEstadísticas descriptivas de las columnas numéricas:")
            print(df.describe().to_markdown(numalign="left", stralign="left"))

            # Mostrar el número de valores únicos para columnas no numéricas
            print("\nValores únicos de columnas no numéricas:")
            for columna in df.columns:
                if not pd.api.types.is_numeric_dtype(df[columna]):
                    # Convertir valores de la columna a listas si son arreglos de NumPy
                    if isinstance(df[col].iloc[0], pd.Series):
                         num_valores_unicos = df[col].nunique()
                    elif isinstance(df[col].iloc[0], dict):
                        num_valores_unicos = df[col].nunique()
                    else:
                        num_valores_unicos = df[col].nunique()
                    print(f"- {columna}: {num_valores_unicos} valores únicos")

            # Generar gráficos para columnas numéricas
            print("\nGráficos para columnas numéricas:")
            for columna in df.select_dtypes(include=['number']).columns:
                plt.figure()
                plt.title(f"Histograma de {columna}")
                sns.histplot(df[col], kde=False)  # Desactivamos la estimación de densidad del kernel
                plt.xlabel(columna)
                plt.ylabel("Frecuencia")
                plt.show()  # Mostrar el gráfico

            # Generar gráficos de dispersión para pares de columnas numéricas
            print("\nGráficos de dispersión para pares de columnas numéricas:")
            num_cols = df.select_dtypes(include=['number']).columns
            if len(num_cols) > 1:
                for i in range(len(num_cols)):
                    for j in range(i + 1, len(num_cols)):
                        col1 = num_cols[i]
                        col2 = num_cols[j]
                        plt.figure()
                        plt.title(f"Dispersión entre {col1} y {col2}")
                        sns.scatterplot(data=df, x=col1, y=col2)
                        plt.xlabel(col1)
                        plt.ylabel(col2)
                        plt.show()  # Mostrar el gráfico
            else:
                print("No hay suficientes columnas numéricas para generar gráficos de dispersión.")

        except Exception as e:
            print(f"Error al procesar el archivo {archivo}: {e}")

# Ejecutar el EDA en el directorio de archivos Parquet
realizar_eda_parquet(parquet_dir)



Análisis del archivo: ../data/parquet_files\metadata_sitios_All.parquet

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025011 entries, 0 to 3025010
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   name              object 
 1   address           object 
 2   gmap_id           object 
 3   description       object 
 4   latitude          float64
 5   longitude         float64
 6   category          object 
 7   avg_rating        float64
 8   num_of_reviews    int64  
 9   price             object 
 10  hours             object 
 11  MISC              object 
 12  state             object 
 13  relative_results  object 
 14  url               object 
dtypes: float64(3), int64(1), object(11)
memory usage: 346.2+ MB

Primeras 5 filas del DataFrame:
| name             | address                                                | gmap_id                               | description   | latitude   | longit

KeyboardInterrupt: 